In [220]:
#!/usr/bin/env python
from functools import reduce
from itertools import permutations
from operator import mul
from os import makedirs
from os.path import isdir, isfile, join
from time import time

import numpy as np
from netCDF4 import Dataset
from numpy import arange
from numpy.random import random_sample

import dask.array as da

In [106]:
chunk = (None, 1)
dimensions = {'0': 10, '1': 1000}
numslices = 100
outputdir = 'slices'
prefix = 'slice.'
serial = False
variables = {}

In [107]:
dims = sorted(dimensions)
variables = {}
n = 0
for i in range(len(dims)):
    for vdims in permutations(dims, i + 1):
        vname = f'v{n}'
        variables[vname] = tuple(vdims)
        n += 1

In [108]:
variables

{'v0': ('0',), 'v1': ('1',), 'v2': ('0', '1'), 'v3': ('1', '0')}

In [5]:
makedirs(outputdir, exist_ok=True)

In [7]:
for nslice in range(numslices):
    fname = join(outputdir, f'{prefix}{nslice}')
    with Dataset(fname, 'w') as fobj:
        fobj.setncattr('file', fname)
        fobj.setncattr('slice', str(nslice))

        for dname in dimensions:
            if dname == "0":
                fobj.createDimension(dname)

            else:
                fobj.createDimension(dname, dimensions[dname])

        for dname in dimensions:
            vobj = fobj.createVariable(dname, 'd', (dname,))
            vobj.setncattr('units', '1')
            vobj.setncattr('comment', f'Coordinate {dname}')
            dlen = dimensions[dname]
            if dname == "0":
                vobj[:] = arange(nslice * dlen, (nslice + 1) * dlen, dtype='d')

            else:
                vobj[:] = arange(dlen, dtype='d')

        fobj.sync()

        vobjs = {}
        for vname in variables:
            vdims = variables[vname]
            vtype = 'f' if len(vdims) > 1 else 'd'
            vobj = fobj.createVariable(vname, vtype, vdims)
            vobj.setncattr('units', '1')
            vobj.setncattr('comment', f'Variable {vname}')
            vobjs[vname] = vobj
        fobj.sync()

        for vname in vobjs:
            vobj = vobjs[vname]
            ndims = len(vobj.dimensions)
            if ndims == 0:
                vobj[:] = 1.0
            elif ndims == 1:
                vobj[:] = arange(dimensions[vobj.dimensions[0]], dtype='d')
            else:
                cname = chunk[0]
                if cname in vobj.dimensions:
                    cdlen = dimensions[cname]
                    csize = chunk[1]
                else:
                    cdlen = 1
                    csize = 1

                nchnks = cdlen // csize + int(cdlen % csize > 0)
                for n in range(nchnks):
                    istart = n * csize
                    iend = cdlen if n == nchnks - 1 else (n + 1) * csize
                    slc = tuple(
                        slice(istart, iend) if d == cname else slice(None) for d in vobj.dimensions
                    )
                    shp = tuple(csize if d == cname else dimensions[d] for d in vobj.dimensions)
                    vobj[slc] = random_sample(shp)
        fobj.sync()

In [8]:
import xarray as xr

In [185]:
d = xr.open_dataset("slices/slice.1")
d

<xarray.Dataset>
Dimensions:  (0: 10, 1: 1000)
Coordinates:
  * 0        (0) float64 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0 18.0 19.0
  * 1        (1) float64 0.0 1.0 2.0 3.0 4.0 ... 995.0 996.0 997.0 998.0 999.0
Data variables:
    v0       (0) float64 ...
    v1       (1) float64 ...
    v2       (0, 1) float32 ...
    v3       (1, 0) float32 ...
Attributes:
    file:     slices/slice.1
    slice:    1

In [109]:
dimensions = {'0': 10, '1': 1000}
dims = sorted(dimensions)
variables = {}
n = 0
for i in range(len(dims)):
    for vdims in permutations(dims, i + 1):
        vname = f'v{n}'
        variables[vname] = tuple(vdims)
        n += 1

In [110]:
variables

{'v0': ('0',), 'v1': ('1',), 'v2': ('0', '1'), 'v3': ('1', '0')}

In [111]:
dims

['0', '1']

In [113]:
d

<xarray.Dataset>
Dimensions:  (0: 10, 1: 1000)
Coordinates:
  * 0        (0) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0
  * 1        (1) float64 0.0 1.0 2.0 3.0 4.0 ... 995.0 996.0 997.0 998.0 999.0
Data variables:
    v0       (0) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0
    v1       (1) float64 ...
    v2       (0, 1) float32 ...
    v3       (1, 0) float32 0.6189122 0.082804225 ... 0.06779086 0.7727069
Attributes:
    file:     slices/slice.0
    slice:    0

In [114]:
for key, item in variables.items():
    print(key, item)

v0 ('0',)
v1 ('1',)
v2 ('0', '1')
v3 ('1', '0')


In [51]:
xr.DataArray(
    data=arange(10, dtype='float64'), name='v0', coords={'0': arange(10, dtype='float')}, dims=('0')
)

<xarray.DataArray 'v0' (0: 10)>
array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
Coordinates:
  * 0        (0) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0

In [115]:
dimensions

{'0': 10, '1': 1000}

In [87]:
data_vars = {}
for name, dim_names in variables.items():
    coords = {}
    for dim in dim_names:
        coords[dim] = arange(
            dimensions[dim],
            dtype='float32',
        )
    data_vars[name] = coords

In [93]:
data_vars['v0']

{'0': array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)}

In [203]:
from functools import reduce
from operator import mul

dset = xr.Dataset()
for data_var, dims in data_vars.items():
    shape = []
    d = tuple(data_vars[data_var].keys())
    for item in d:
        shape.append(dimensions[item])

    shape = tuple(shape)
    nelements = reduce(mul, shape, 1)
    print(nelements, shape)
    dset[data_var] = xr.DataArray(
        arange(nelements, dtype='float').reshape(shape), coords=data_vars[data_var], dims=d
    )
    dset[data_var].attrs['units'] = '1'
    dset[data_var].attrs['comment'] = f'Variable {data_var}'

for coord in dset.coords:
    dset.coords[coord].attrs['units'] = '1'
    dset.coords[coord].attrs['comment'] = f'Coordinate {coord}'

10 (10,)
1000 (1000,)
10000 (10, 1000)
10000 (1000, 10)


In [207]:
dims = sorted(dimensions)
variables = {}
n = 0
for i in range(len(dims)):
    for vdims in permutations(dims, i + 1):
        vname = f'v{n}'
        variables[vname] = tuple(vdims)
        n += 1

In [208]:
dims

['0', '1']

In [209]:
variables

{'v0': ('0',), 'v1': ('1',), 'v2': ('0', '1'), 'v3': ('1', '0')}

In [210]:
dimensions

{'0': 10, '1': 1000}

In [211]:
numslices

100

In [221]:
for nslice in range(numslices):
    path = f"{outputdir}/slice{str(nslice)}.nc"

    data_vars = {}
    for var_name, var_dims in variables.items():
        coords = {}
        for dim in var_dims:
            dlen = dimensions[dim]
            if dim == "0":
                coords[dim] = np.arange(nslice * dlen, (nslice + 1) * dlen, dtype='d')
            else:
                coords[dim] = np.arange(dlen, dtype='d')
    print(coords['1'])

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181.
 182. 183. 184. 185. 186. 187. 188. 189. 190. 191. 192. 193. 194. 195.
 196. 